In [2]:
"""
original 데이터셋을 사용하는 게 의미 있는가?가 궁금해서
동일 model에서 train만 사용, original만 사용 roc_auc 비교

MEHRAN 의 notebook의 첫 번째 submission 부분만 사용
(더 해보려 했는데 다른 문제가 좀 있어서... 시간되면 더 비교해 보기로)

feature engineering: log1p 적용, 혼자 경향이 다른 'l'만 1 - 'l' 적용
"""

"\noriginal 데이터셋을 사용하는 게 의미 있는가?가 궁금해서\n동일 model에서 train만 사용, original만 사용 roc_auc 비교\n\nMEHRAN 의 notebook의 첫 번째 submission 부분만 사용\n(더 해보려 했는데 다른 문제가 좀 있어서... 시간되면 더 비교해 보기로)\n\nfeature engineering: log1p 적용, 혼자 경향이 다른 'l'만 1 - 'l' 적용\n"

In [27]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [28]:
train = pd.read_csv('input/train.csv', index_col='id')
test = pd.read_csv('input/test.csv', index_col='id')
original = pd.read_csv('input/jm1.csv')

In [29]:
def check_missing(df, name):
    missing_values = df.isnull().sum()
    print('Missing Value in ', name, ': ', missing_values[missing_values > 0])
    print('Duplicates in ', name, ': ', df.duplicated().sum())
    print(df.dtypes)
    
check_missing(train, "train")
check_missing(test, "test")
check_missing(original, "original")

Missing Value in  train :  Series([], dtype: int64)
Duplicates in  train :  0
loc                  float64
v(g)                 float64
ev(g)                float64
iv(g)                float64
n                    float64
v                    float64
l                    float64
d                    float64
i                    float64
e                    float64
b                    float64
t                    float64
lOCode                 int64
lOComment              int64
lOBlank                int64
locCodeAndComment      int64
uniq_Op              float64
uniq_Opnd            float64
total_Op             float64
total_Opnd           float64
branchCount          float64
defects                 bool
dtype: object
Missing Value in  test :  Series([], dtype: int64)
Duplicates in  test :  0
loc                  float64
v(g)                 float64
ev(g)                float64
iv(g)                float64
n                    float64
v                    float64
l                   

In [30]:
# original 을 좀 정리하면..

# 1. duplicate 제거
original.drop_duplicates(inplace=True)

original.dtypes # original 의 일부 데이터가 ? 이다 

# 2. float 이 아닌 Object인 feature들 정리 - 5개의 row가 ?를 갖고 있다 -> 해당 row 제거
#for column in ['uniq_Op', 'uniq_Opnd', 'total_Op','total_Opnd', 'branchCount']:
#    print(len(original[original[column] == '?']), original[original[column] == '?'].index)

for column in ['uniq_Op', 'uniq_Opnd', 'total_Op','total_Opnd', 'branchCount']:
    original.drop(original[original[column] == '?'].index, inplace=True)
    original[column] = original[column].astype(float)

In [31]:
## features: 모두 사용
features = [f for f in train.columns.tolist() if f != 'defects']
print(features)

for f in features:
    train[f] = np.log1p(train[f])
    test[f] = np.log1p(test[f])
    original[f] = np.log1p(original[f])
    
# 'l'은 다른 feature들과 다르게 defects와 음의 상관관계를 갖고 있으므로 이를 역전시켜 준다
train['l'] = 1 - train['l']
test['l'] = 1 - test['l']
original['l'] = 1 - original['l']

['loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e', 'b', 't', 'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount']


In [32]:
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import QuantileTransformer

In [36]:
# 1 
#   QuantileTransformer (normalization)
#   GuassianNB
from sklearn.metrics import roc_auc_score, roc_curve, auc

def QuantileTransformer_GuassianNB(X, y, label): 
    pipeline = make_pipeline(QuantileTransformer(output_distribution='normal'),
                            GaussianNB())
    pipeline.fit(X, y)
    cross_val_score(pipeline, X, y, scoring='roc_auc', cv=10).mean()
    fpr, tpr, _ = roc_curve(y, pipeline.predict_proba(X)[:,1])
    roc_auc = auc(fpr, tpr)
    
    print(label + ': ' + str(roc_auc))

# only with train data
X1 = train[features]    
y1 = train['defects']
QuantileTransformer_GuassianNB(X1, y1, "train-only")

# with train/original data
concat = pd.concat([train,original], axis=0)
concat.reset_index(inplace=True, drop=True)
X2 = concat[features]
y2 = concat['defects']
QuantileTransformer_GuassianNB(X2, y2, "train+original")

train-only: 0.7571282304482544
train+original: 0.7491242595402765
